In [263]:
import logging
import sys
import datetime

from icecream import ic
from tqdm import tqdm

In [612]:

def time_now():
  
    return f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} |"
#.strftime('%Y-%m-%d %H:%M:%S')
ic.configureOutput(prefix=time_now)
ic.configureOutput(includeContext=True) 

https://github.com/jhljx/GKT 

https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/aggr/lstm.html

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt;
np.random.seed(42);

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:matplotlib:matplotlib data path: C:\Users\mpathirana\Anaconda3\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\mpathirana\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_frozen_importlib_external', '_io', 'marshal', 'nt', '_thread', '_weakref', 'winreg', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', 'encodings.cp1252', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'ntpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'pywin32_bootstrap', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_heapq', 'heapq', 'itertools', 'repr

In [3]:
from sklearn.metrics import roc_auc_score;
from sklearn.metrics import classification_report;

In [4]:
pd.set_option('display.max_columns', None);

In [5]:
df=pd.read_pickle('appmath.pkl')[['graph_id', 'user_id', 
       'goal_id',
        'interaction_end_time',
       'learning_objective_name', 'atom_id', 
       'correct', 'time_spent_answering_s', 'time_spent_on_instruction_s',
       'goal_progress',
       'target_status_and_progress', 'prev_concept_narrative',
       'duration_s', 'is_target']]  

In [6]:
df2=pd.read_csv('prereq_edges.csv')

In [7]:
G=nx.Graph()

for _,i in df2.iterrows():    
       
        G.add_edge(i['source_lo_title'],i['dest_lo_title'])
        
subgraphs=[G.subgraph(i) for i in nx.connected_components(G)]

for num,sub in enumerate(subgraphs):
    
    nx.set_node_attributes(sub,num,'subGraphId')

nodeSubGraphId={}
for g in subgraphs:
    for node in g.nodes(data=True):
         nodeSubGraphId[node[0]]= node[1]['subGraphId']
            
subgraphsNodeCountDict={ num: g.number_of_nodes() for num,g in  enumerate(subgraphs)}

In [8]:
for col in df.columns:
    if df[col].dtype.name=='category':
        df[col]=df[col].astype(str)

https://colab.research.google.com/drive/1jDiuseAdtZCYbAX2IRv2HU8zG5585bRi?usp=sharing#scrollTo=GQ9SKdThF8Bq

https://anuradhawick.com/software-tools/

In [9]:
df['loGraphID']=df.learning_objective_name.apply(lambda x: nodeSubGraphId.get(x) )

In [10]:
difficulty=df[~df.correct.isna()].groupby('atom_id')['correct'].mean().reset_index().rename(columns={'correct':'difficulty'})

In [11]:
df=df.merge(difficulty,how='left',on='atom_id')

In [12]:
#loIndex={lo:i for i,lo in enumerate( df.learning_objective_name.unique())}
#df['loIndex']=df['learning_objective_name'].apply(lambda x: loIndex[x])

In [13]:
def atomClassifier(x):
    if x==None:
        return 'learningMaterial'
    else:
        return 'question'

In [14]:
df['atom_type']=df.correct.apply(lambda x:atomClassifier(x) )

In [15]:
df['correctBinary']=df.correct*1
df.atom_id=df.atom_id.astype(str)


In [23]:
df2['graph_id']=df2.apply(lambda x: nodeSubGraphId.get(x['source_lo_title']) if nodeSubGraphId.get(x['source_lo_title'])!=None else nodeSubGraphId.get(x['dest_lo_title']),axis=1)

In [84]:
unique_lo_graph_id_dic={}
for i in df2.graph_id.unique():
    unique_lo=np.unique(np.concatenate((df2[df2.graph_id==i]['source_lo_title'].unique(),((df2[df2.graph_id==i]['dest_lo_title'].unique())))))
    unique_lo_graph_id_dic[i]={ item:int(num)  for  num,item in enumerate(unique_lo)}
    df2.loc[df2.graph_id==i,'source_index' ]=df2.apply(lambda x: unique_lo_graph_id_dic.get(i).get(x['source_lo_title']),axis=1)
    df2.loc[df2.graph_id==i,'dest_index' ]=df2.apply(lambda x: unique_lo_graph_id_dic.get(i).get(x['dest_lo_title']),axis=1)

In [87]:
df2.source_index=df2.source_index.astype(int)
df2.dest_index=df2.dest_index.astype(int)

In [102]:
df=df[~df.loGraphID.isna()]

In [103]:
df['loIndex']=df.apply(lambda x: unique_lo_graph_id_dic[x['loGraphID']].get(x['learning_objective_name'] ),axis=1)

C:\Users\mpathirana\AppData\Local\Temp\ipykernel_9232\447430040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loIndex']=df.apply(lambda x: unique_lo_graph_id_dic[x['loGraphID']].get(x['learning_objective_name'] ),axis=1)


In [141]:
df2[df2.graph_id==3].max()

source_lo_id                    eda7ab6a-8070-4fd8-8387-da17a40fd99e
dest_lo_id                      eda7ab6a-8070-4fd8-8387-da17a40fd99e
source_lo_title    Write biconditional statements in symbolic for...
dest_lo_title      Write biconditional statements in symbolic for...
graph_id                                                           3
source_index                                                      30
dest_index                                                        30
dtype: object

In [331]:
loGraphID=26

In [332]:
loEdgeMapping=df2[df2['graph_id']==loGraphID][['source_index','dest_index']]
loEdgeIndex=loEdgeMapping.values.transpose()

In [333]:
def user_dataset(df,df2,user,goal):
    
    '''
    df - df should be filtered by goal id
    
    edges
    -------
    lo -> atom
    atom -> atom
    lo -> learning materials (lm)
    atom -> lm
    lm -> atom
    lm -> lm
    lm/atom -> target
    
    node / features
    ---------------
    
    lo - radnom number
    atom - dificulty , number of attempt, correctness
    lm - time spend 
        
    '''    
    df=df[(df.loGraphID==goal) & (df.user_id==user )   ].reset_index(drop=True)
    userDf=df.sort_values(by='interaction_end_time')
    
    
    loEdgeMapping=df2[df2['graph_id']==goal][['source_index','dest_index']]
    loEdgeIndex=loEdgeMapping.values.transpose()
    
    
    atomIndex=np.arange(userDf[userDf.atom_type=='question'].atom_id.shape[0])
    lmIndex=np.arange(userDf[userDf.atom_type=='learningMaterial'].atom_id.shape[0])
    
    # assisgning questions and learningMaterials index seperatly in sinlge columns 'atom_index'
    userDf.loc[userDf.atom_type=='question','atomIndex']=atomIndex
    userDf.loc[userDf.atom_type=='learningMaterial','atomIndex']=lmIndex
    
    #lo-atom
    loAtomEdgeIndex=userDf[userDf.atom_type=='question'][['loIndex','atomIndex']].values.transpose()
    #lo-lm
    loLmEdgeIndex=userDf[userDf.atom_type=='learningMaterial'][['loIndex','atomIndex']].values.transpose()
    
    userDf['attempt_count']=userDf.groupby(by=['atom_id']).cumcount()+1
    
    userDf['atom_type_shift']=userDf.atom_type.shift(-1)
    userDf['atomIndex_shift']=userDf.atomIndex.shift(-1)
    
    #atom features
    atomFeatures = userDf[userDf.atom_type=='question'][['difficulty']].to_numpy()
    #learning materials features
    lmFeatures = userDf[userDf.atom_type=='learningMaterial'][['duration_s']].to_numpy()
    #learning objectives
    loFeatures=np.ones(loEdgeIndex.max()+1).reshape(-1,1)
    #target
    target=userDf[userDf.atom_type=='question']['correctBinary'].astype(int).to_numpy()
    
    userDf_shift= userDf[~userDf.atomIndex_shift.isna()].copy()  
    
    #atom-atom
    atomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #atom-lm
    atomLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-atom
    LmAtomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-lm
    LmLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    featuresDict={'atom':atomFeatures,'lm':lmFeatures,'lo':loFeatures}
    edgesDict={'lo_atom':loAtomEdgeIndex,'lo_lm':loLmEdgeIndex,'lo':loEdgeIndex,'atom': atomEdgeIndex,'atom_lm':atomLmEdgeIndex,'lm_atom':LmAtomEdgeIndex,'lm_lm':LmLmEdgeIndex}
    
    return featuresDict,edgesDict,target
    
    
    
    
    

In [334]:
from torch_geometric.data import HeteroData
import torch 


In [335]:
import torch_geometric.transforms as T

In [336]:
def generate_data(features,edges,y):  
    data = HeteroData()
    #node features
    data['atom'].x=torch.tensor(features['atom']).float()
    data['lo'].x= torch.tensor(features['lo']).float()
    data['lm'].x= torch.tensor(features['lm']).float()
    
    #edge index
    data['lo','lo_atom','atom'].edge_index=torch.from_numpy(edges['lo_atom']).long()
    data['lo','lo_lo','lo'].edge_index= torch.from_numpy(edges['lo']).long()
    data['atom','atom_atom','atom'].edge_index=torch.from_numpy(edges['atom']).long()
    data['lo','lo_lm','lm'].edge_index=torch.from_numpy(edges['lo_lm']).long()
    data['atom','atom_lm','lm'].edge_index=torch.from_numpy(edges['atom_lm']).long()
    data['lm','atom_lm','atom'].edge_index=torch.from_numpy(edges['lm_atom']).long()
    data['lm','lm_lm','lm'].edge_index=torch.from_numpy(edges['lm_lm']).long()

    data['atom'].y=torch.from_numpy(y).long()


    #split=T.RandomNodeSplit(num_test =5)
    #data=split(data)

    size=y.shape[0]
    #num_test_nodes =int(y.shape[0]*.001)
    #some_indices = torch.randperm(size)[:num_test_nodes]
    some_indices = torch.randperm(size)[0]


    test_mask = torch.zeros(y.shape[0], dtype=torch.bool)
    test_mask[some_indices] = True
    data['atom'].test_mask = test_mask

    train_mask = torch.ones(y.shape[0], dtype=torch.bool)
    train_mask[some_indices] = False
    data['atom'].train_mask = train_mask

    normalize=T.NormalizeFeatures()
    data=normalize(data)
    return data

In [337]:
features, edges,target =user_dataset(df,df2,'0012ff2b-871a-45d6-8e33-804f7464fa0a',3 )

In [338]:
data=generate_data(features, edges,target)

In [339]:
users=df[(df.loGraphID==loGraphID)].user_id.unique()

In [340]:
usersGraphDataList=[]
min_atom_count=5
max_atom_count=500
for user in tqdm(users): 
    try:
        user_atom_count=df[(df.loGraphID==loGraphID) & (df.user_id==user) & (df.atom_type=='question')].shape[0]
        if user_atom_count>min_atom_count and user_atom_count<=max_atom_count:
            features, edges,target =user_dataset(df,df2,user,loGraphID )
            data=generate_data(features, edges,target)
            usersGraphDataList.append(data)
    except:
        pass
    

100%|████████████████████████████████████████████████████████████████████████████████| 274/274 [03:24<00:00,  1.34it/s]


In [341]:
print('data.has_isolated_nod : {}'.format(data.has_isolated_nodes()))
print('data.has_self_loops   : {}'.format(data.has_self_loops()))
print('data.is_undirected    : {}'.format(data.is_directed()))

data.has_isolated_nod : False
data.has_self_loops   : False
data.is_undirected    : True


In [342]:
import torch_geometric
from torch_geometric.nn import GCNConv 
from torch_geometric.nn import SAGEConv, to_hetero ,LSTMAggregation
import  torch.nn as nn
import torch.nn.functional as F
import torch 
from torch_geometric.loader import DataLoader

In [343]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels,requires_grad=False)
        self.actfn1=nn.ReLU()
        
        self.conv2 = SAGEConv((-1, -1), hidden_channels,requires_grad=False)
        self.actfn2=nn.ReLU()
        
        self.conv3 = SAGEConv((-1, -1), hidden_channels,requires_grad=False)
        self.actfn3=nn.ReLU()
        
        #self.linear=nn.Linear(hidden_channels,hidden_channels)
        
        #self.lstm = nn.LSTM(input_size=hidden_channels, hidden_size=50,num_layers=1, batch_first=True)
        
        self.out=nn.Linear(hidden_channels,out_channels)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x, edge_index):
        # First Message Passing layer
        x = self.actfn1(self.conv1(x, edge_index))
        x= F.dropout(x,p=.1,training=self.training)
        
        # Second Message Passing layer
        x = self.actfn2(self.conv2(x, edge_index))
        x= F.dropout(x,p=.1,training=self.training)
        
        # Third Message Passing layer  
        #x = self.actfn3(self.conv3(x, edge_index))
        #x= F.dropout(x,p=.25,training=self.training)
        #x = self.linear(x)
        #x, _ = self.lstm(x)
       
        
        # Out layer
        x=self.sigmoid( self.out(x))
        
        return x


#model_1 = GNN(hidden_channels=64, out_channels=2)
#model_1 = to_hetero(model_1, data.metadata(), aggr='sum')

In [344]:
usersGraphDataList
def batch_generator(usersGraphDataList):
    
    np.random.shuffle(usersGraphDataList)
    test_size=int(len(usersGraphDataList)*.2)
    test_data=usersGraphDataList[:test_size]
    train_data=usersGraphDataList[test_size:]
    
    return test_data,train_data
    

In [349]:
model_1 = GNN(hidden_channels=200, out_channels=2)
model_1 = to_hetero(model_1, data.metadata(), aggr='mean')


#model_2 = GAT(hidden_channels=100, out_channels=2)
#model_2 = to_hetero(model_2, data.metadata(), aggr='mean')


model=model_1
#dataLoader=list(DataLoader(usersGraphDataList,batch_size=80))
test_data,train_data=batch_generator(usersGraphDataList)
learning_rate=0.01
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion=torch.nn.CrossEntropyLoss()
train_acc_mean_list=[]
test_acc_mean_list=[]
loss_list=[]
best_score=0

start_training=True
if start_training:
    for epcoh in range(7):
        train_acc_list=[]
        test_acc_list=[]
        loss_list=[]
        for num, data in enumerate(tqdm(train_data)):
            model.train()
            optimizer.zero_grad()
            #data.x = torch.from_numpy(data.x_dict)
            #data.edge_index = torch.from_numpy(data.edge_index_dict)
            out=model(data.x_dict, data.edge_index_dict)  # model1
            #out = model(data.x, data.edge_index)
            #BCELoss -> loss=criterion(out['atom'][data['atom'].train_mask].argmax(dim=1).float().requires_grad_(True),data['atom'].y[data['atom'].train_mask].float())
            #loss=criterion(out['atom'][data['atom'].train_mask].argmax(dim=1).float().requires_grad_(True),data['atom'].y[data['atom'].train_mask].float())
            loss=criterion(out['atom'],data['atom'].y)
            #loss=criterion(out['atom'].argmax(dim=1).float().requires_grad_(True),data['atom'].y.float())
            loss.backward()

            optimizer.step()
            pred=out['atom'].argmax(dim=1)
            train_correct=pred==data['atom'].y
            train_acc=int(train_correct.sum())/data['atom'].y.shape[0]
            train_acc_list.append(train_acc)



        #test_acc_mean_list.append(np.mean(test_acc_list))
            if epcoh==0 and num ==0:
                best_score=loss
            elif best_score<loss:
                best_score=loss
                best_model = model
            loss_list.append(loss.item())
        model.eval()
        model_performance='epoch : {} , loss :{:.4f} , accuracy : {:.4f} \n'.format(epcoh+1,np.mean(loss_list),np.mean(train_acc_list))
        ic(model_performance);



100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:07<00:00, 19.56it/s]
2023-10-15 16:11:56 |model_performance: 'epoch : 1 , loss :0.6338 , accuracy : 0.6564 
                                         '
100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:17<00:00,  8.84it/s]
2023-10-15 16:12:13 |model_performance: 'epoch : 2 , loss :0.6202 , accuracy : 0.6695 
                                         '
100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:08<00:00, 18.65it/s]
2023-10-15 16:12:21 |model_performance: 'epoch : 3 , loss :0.6167 , accuracy : 0.6757 
                                         '
100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:08<00:00, 17.23it/s]
2023-10-15 16:12:30 |model_performance: 'epoch : 4 , loss :0.6155 , accuracy : 0.6811 
                                         '


In [353]:
test_score=[]
target=[]
pred_prob=[]
for num, data in enumerate(test_data):
    try:
        out=model(data.x_dict, data.edge_index_dict)
        pred=out['atom'].argmax(dim=1)
        test_correct=pred[data['atom'].test_mask]==data['atom'].y[data['atom'].test_mask]
        test_acc=int(test_correct.sum())/int(data['atom'].test_mask.sum())
        print('Test case : {} , nodes : {} , accuracy : {:.4f} '.format(num+1,data['atom'].x.shape[0],test_acc))
        target.append(data['atom'].y[data['atom'].test_mask].item())
        pred_prob.append(out['atom'][data['atom'].test_mask].detach()[0].tolist()[1])
        test_score.append(test_acc)
    except:
        pass
print('Mean accuracy : {} , AUC : {}'.format(np.mean(test_score),roc_auc_score(np.array(target),pred_prob)))

Test case : 1 , nodes : 33 , accuracy : 1.0000 
Test case : 2 , nodes : 26 , accuracy : 1.0000 
Test case : 3 , nodes : 11 , accuracy : 1.0000 
Test case : 4 , nodes : 14 , accuracy : 1.0000 
Test case : 5 , nodes : 36 , accuracy : 0.0000 
Test case : 6 , nodes : 61 , accuracy : 0.0000 
Test case : 7 , nodes : 16 , accuracy : 0.0000 
Test case : 8 , nodes : 13 , accuracy : 1.0000 
Test case : 9 , nodes : 17 , accuracy : 0.0000 
Test case : 10 , nodes : 23 , accuracy : 1.0000 
Test case : 11 , nodes : 20 , accuracy : 1.0000 
Test case : 12 , nodes : 14 , accuracy : 0.0000 
Test case : 13 , nodes : 20 , accuracy : 0.0000 
Test case : 14 , nodes : 28 , accuracy : 0.0000 
Test case : 15 , nodes : 19 , accuracy : 1.0000 
Test case : 16 , nodes : 8 , accuracy : 1.0000 
Test case : 17 , nodes : 82 , accuracy : 0.0000 
Test case : 18 , nodes : 71 , accuracy : 0.0000 
Test case : 19 , nodes : 28 , accuracy : 1.0000 
Test case : 20 , nodes : 9 , accuracy : 0.0000 
Test case : 21 , nodes : 58 , a

In [210]:
#user_dataset(df,df2,'0012ff2b-871a-45d6-8e33-804f7464fa0a',3 )
df.columns

Index(['graph_id', 'user_id', 'goal_id', 'interaction_end_time',
       'learning_objective_name', 'atom_id', 'correct',
       'time_spent_answering_s', 'time_spent_on_instruction_s',
       'goal_progress', 'target_status_and_progress', 'prev_concept_narrative',
       'duration_s', 'is_target', 'loGraphID', 'difficulty', 'loIndex',
       'atom_type', 'correctBinary'],
      dtype='object')

In [368]:
dft=df[(df.loGraphID==loGraphID)]

In [369]:
#dft.groupby('user_id')['atom_id'].count()

In [551]:
user_t='05f0698d-d380-42be-ae43-5e87301254ef'

In [635]:
df_tu=dft[dft.user_id==user_t].sort_values('interaction_end_time').reset_index(drop=True)

In [636]:
df_tu_list=[]
question_count=0
for row_id,row in df_tu.iterrows():
    if row['atom_type'] =='question':
        question_count =question_count+1
        if question_count>3:
            df_tu_list.append(df_tu.iloc[:row_id+1])

In [637]:
len(df_tu_list)


45

def user_dataset_with_target_node(df,df2,user,goal,loEdgeIndex):
    
    '''
    df - df should be filtered by goal id
    
    edges
    -------
    lo -> atom
    atom -> atom
    lo -> learning materials (lm)
    atom -> lm
    lm -> atom
    lm -> lm
    lm/atom -> target
    
    node / features
    ---------------
    
    lo - radnom number
    atom - dificulty , number of attempt, correctness
    lm - time spend 
        
    '''    
    df=df[(df.loGraphID==goal) & (df.user_id==user )   ].reset_index(drop=True)
    userDf=df.sort_values(by='interaction_end_time')
    
    atomIndex=np.arange(userDf[userDf.atom_type=='question'].atom_id.shape[0])
    lmIndex=np.arange(userDf[userDf.atom_type=='learningMaterial'].atom_id.shape[0])
    
    # assisgning questions and learningMaterials index seperatly in sinlge columns 'atom_index'
    userDf.loc[userDf.atom_type=='question','atomIndex']=atomIndex
    userDf.loc[userDf.atom_type=='learningMaterial','atomIndex']=lmIndex
    
    #lo-atom
    loAtomEdgeIndex=userDf[userDf.atom_type=='question'][['loIndex','atomIndex']].values.transpose()
    #lo-lm
    loLmEdgeIndex=userDf[userDf.atom_type=='learningMaterial'][['loIndex','atomIndex']].values.transpose()
    userDf['atom_type_shift']=userDf.atom_type.shift(-1)
    userDf['atomIndex_shift']=userDf.atomIndex.shift(-1)
    
    #atom features
    atomFeatures = userDf[userDf.atom_type=='question'][['difficulty','correctBinary']].to_numpy()
    if atomFeatures.size==0 :
        ic('no_attoms')
        return None
    else:
        featuresDict={}
        featuresDict['atom']=atomFeatures
        #learning materials features
        lmFeatures = userDf[userDf.atom_type=='learningMaterial'][['duration_s']].to_numpy()
        if  lmFeatures.size>0 :
            featuresDict['lm']=lmFeatures
        #learning objectives
        loFeatures=np.ones(loEdgeIndex.max()+1).reshape(-1,1)
        featuresDict['lo']=loFeatures
        
        #target
        #target=userDf[userDf.atom_type=='question']['correctBinary'].astype(int).to_numpy()
        if userDf.shape[0]>1:
            target_atom=userDf.copy().iloc[-1,:]
        else:
            target_atom= None
        userDf_shift= userDf[~userDf.atomIndex_shift.isna()].copy() 

        edgesDict={}
        #atom-atom
        atomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
        if atomEdgeIndex.size >0:
            edgesDict['atom']=atomEdgeIndex
        #atom-lmatomLmEdgeIndex
        atomLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
        if atomLmEdgeIndex.size>0:
            edgesDict['atom_lm']=atomLmEdgeIndex
        #lm-atom
        LmAtomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
        if LmAtomEdgeIndex.size>0:
            edgesDict['lm_atom']=LmAtomEdgeIndex
        #lm-lm
        LmLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
        if LmLmEdgeIndex.size>0:
            edgesDict['lm_lm']=LmLmEdgeIndex

        if  loLmEdgeIndex.size>0:
            edgesDict['lo_lm']=loLmEdgeIndex

        if  loAtomEdgeIndex.size>0:
            edgesDict['lo_atom']=loAtomEdgeIndex

        #edgesDict={'lo_atom':loAtomEdgeIndex,'lo_lm':loLmEdgeIndex,'lo':loEdgeIndex,'atom': atomEdgeIndex,'atom_lm':atomLmEdgeIndex,'lm_atom':LmAtomEdgeIndex,'lm_lm':LmLmEdgeIndex}
    
        return featuresDict,edgesDict,target_atom,userDf
    
    
    
    
    

In [675]:
def user_dataset_with_target_node(userDf,df2,user,goal,loEdgeIndex):
    
    '''
    df - df should be filtered by goal id
    
    edges
    -------
    lo -> atom
    atom -> atom
    lo -> learning materials (lm)
    atom -> lm
    lm -> atom
    lm -> lm
    lm/atom -> target
    
    node / features
    ---------------
    
    lo - radnom number
    atom - dificulty ,  correctness
    lm - time spend 
    '''
    userDf.loc[userDf.shape[0]-1,'atom_type']='target'
    
    atomIndex=np.arange(userDf[userDf.atom_type=='question'].atom_id.shape[0])
    lmIndex=np.arange(userDf[userDf.atom_type=='learningMaterial'].atom_id.shape[0])
    
    # assisgning questions and learningMaterials index seperatly in sinlge columns 'atom_index'
    userDf.loc[userDf.atom_type=='question','atomIndex']=atomIndex
    userDf.loc[userDf.atom_type=='learningMaterial','atomIndex']=lmIndex
    userDf.loc[userDf.atom_type=='target','atomIndex']=0
    
    #lo-atom
    loAtomEdgeIndex=userDf[userDf.atom_type=='question'][['loIndex','atomIndex']].values.transpose()
    #lo-lm
    loLmEdgeIndex=userDf[userDf.atom_type=='learningMaterial'][['loIndex','atomIndex']].values.transpose()
    #lo-target
    loTargetEdgeIndex=userDf[userDf.atom_type=='target'][['loIndex','atomIndex']].values.transpose()
    
    userDf['atom_type_shift']=userDf.atom_type.shift(-1)
    userDf['atomIndex_shift']=userDf.atomIndex.shift(-1)
    
    featuresDict={}
    #atom features
    atomFeatures = userDf[userDf.atom_type=='question'][['difficulty','correctBinary']].astype(float).to_numpy()
    featuresDict['atom']=atomFeatures
    #learning materials features
    lmFeatures = userDf[userDf.atom_type=='learningMaterial'][['duration_s']].to_numpy()
    if  lmFeatures.size>0 :
            featuresDict['lm']=lmFeatures
            
    #learning objectives
    loFeatures=np.ones(loEdgeIndex.max()+1).reshape(-1,1)
    featuresDict['lo']=loFeatures
    
    #target features
    targetFeatures = userDf[userDf.atom_type=='target'][['difficulty']].to_numpy()
    featuresDict['target']=targetFeatures
    targetCorrectness=userDf[userDf.atom_type=='target'][['correctBinary']].astype(float).to_numpy()
    
    userDf_shift= userDf[~userDf.atomIndex_shift.isna()].copy() 

    edgesDict={}
    #atom-atom
    atomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    if atomEdgeIndex.size >0:
        edgesDict['atom']=atomEdgeIndex
    #atom-lmatomLmEdgeIndex
    atomLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    if atomLmEdgeIndex.size>0:
        edgesDict['atom_lm']=atomLmEdgeIndex
    #lm-atom
    LmAtomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    if LmAtomEdgeIndex.size>0:
        edgesDict['lm_atom']=LmAtomEdgeIndex
    #lm-lm
    LmLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    if LmLmEdgeIndex.size>0:
        edgesDict['lm_lm']=LmLmEdgeIndex
    
    #lm-target
    LmTargetEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='target')][['atomIndex','atomIndex_shift']].values.transpose()
    if LmTargetEdgeIndex.size>0:
        edgesDict['lm_target']=LmTargetEdgeIndex
    
    #atom-target
    atomTargetEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='target')][['atomIndex','atomIndex_shift']].values.transpose()
    
    edgesDict['atom_target']=atomTargetEdgeIndex

    if  loLmEdgeIndex.size>0:
        edgesDict['lo_lm']=loLmEdgeIndex

    if  loAtomEdgeIndex.size>0:
        edgesDict['lo_atom']=loAtomEdgeIndex
    
    edgesDict['lo_target']=loTargetEdgeIndex
    edgesDict['lo']=loEdgeIndex
    return featuresDict,edgesDict,targetCorrectness,userDf

In [676]:
def preprocess(df,df2,user,goal):
        loEdgeMapping=df2[df2['graph_id']==goal][['source_index','dest_index']]
        loEdgeIndex=loEdgeMapping.values.transpose()
      
        featuresDict,edgesDict,targetCorrectness,userDf=user_dataset_with_target_node(df,df2,user,goal,loEdgeIndex)
        
        return featuresDict,edgesDict,targetCorrectness,userDf
    
            

In [677]:
featuresDict,edgesDict,targetCorrectness,userDf=preprocess(df_tu_list[1].copy(),df2,user_t,loGraphID) 

In [678]:
df_tu_list[1]

,graph_id,user_id,goal_id,interaction_end_time,learning_objective_name,atom_id,correct,time_spent_answering_s,time_spent_on_instruction_s,goal_progress,target_status_and_progress,prev_concept_narrative,duration_s,is_target,loGraphID,difficulty,loIndex,atom_type,correctBinary
0,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:37:45.918,Estimate a value by rounding a whole number,c75b2f7a-396c-486d-920e-8223160cf5df,None,0.000000,147.684006,NaN,NaN,NONE,147.684006,True,26.0,NaN,1,learningMaterial,NaN
1,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:41:58.962,Estimate a value by rounding a whole number,3d04bf7a-927d-4e05-bab6-4e6386c37aa2,True,244.843994,8.200000,0.281250,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,253.043991,True,26.0,0.511905,1,question,1
2,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:43:35.053,Estimate a value by rounding a whole number,e44f88be-6916-4100-9edc-4cff9f825895,True,90.023003,5.801000,0.312500,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,95.824005,True,26.0,0.594340,1,question,1
3,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:45:31.900,Estimate a value by rounding a decimal,18e1f313-ab97-4a70-84f4-0a27047f83db,None,0.000000,116.614998,NaN,NaN,NONE,116.614998,True,26.0,NaN,0,learningMaterial,NaN
4,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:46:47.469,Estimate a value by rounding a decimal,db93d715-4c59-4a94-8847-22b911d86914,False,177.985001,14.199000,0.318522,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,192.184006,True,26.0,0.764706,0,question,0
5,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:47:09.892,Estimate a value by rounding a decimal,dcbc8d4d-648c-4877-b691-6020cf9c6c13,True,7.013000,15.161000,0.349156,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,22.174000,True,26.0,0.636364,0,question,1
6,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:48:41.906,Estimate a value by rounding a whole number,b6be5c36-c782-4f99-9e26-eead67c02682,False,85.869003,5.902000,0.373917,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,91.771004,True,26.0,0.111111,1,question,0


In [679]:
userDf

,graph_id,user_id,goal_id,interaction_end_time,learning_objective_name,atom_id,correct,time_spent_answering_s,time_spent_on_instruction_s,goal_progress,target_status_and_progress,prev_concept_narrative,duration_s,is_target,loGraphID,difficulty,loIndex,atom_type,correctBinary,atomIndex,atom_type_shift,atomIndex_shift
0,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:37:45.918,Estimate a value by rounding a whole number,c75b2f7a-396c-486d-920e-8223160cf5df,None,0.000000,147.684006,NaN,NaN,NONE,147.684006,True,26.0,NaN,1,learningMaterial,NaN,0.0,question,0.0
1,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:41:58.962,Estimate a value by rounding a whole number,3d04bf7a-927d-4e05-bab6-4e6386c37aa2,True,244.843994,8.200000,0.281250,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,253.043991,True,26.0,0.511905,1,question,1,0.0,question,1.0
2,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:43:35.053,Estimate a value by rounding a whole number,e44f88be-6916-4100-9edc-4cff9f825895,True,90.023003,5.801000,0.312500,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,95.824005,True,26.0,0.594340,1,question,1,1.0,learningMaterial,1.0
3,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:45:31.900,Estimate a value by rounding a decimal,18e1f313-ab97-4a70-84f4-0a27047f83db,None,0.000000,116.614998,NaN,NaN,NONE,116.614998,True,26.0,NaN,0,learningMaterial,NaN,1.0,question,2.0
4,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:46:47.469,Estimate a value by rounding a decimal,db93d715-4c59-4a94-8847-22b911d86914,False,177.985001,14.199000,0.318522,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,192.184006,True,26.0,0.764706,0,question,0,2.0,question,3.0
5,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:47:09.892,Estimate a value by rounding a decimal,dcbc8d4d-648c-4877-b691-6020cf9c6c13,True,7.013000,15.161000,0.349156,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,22.174000,True,26.0,0.636364,0,question,1,3.0,target,0.0
6,076e1a0e-669c-40c4-a23c-7ac2bbaa05cf,05f0698d-d380-42be-ae43-5e87301254ef,845d5900-077b-4bb5-b858-cce66cafbd70,2022-08-19 15:48:41.906,Estimate a value by rounding a whole number,b6be5c36-c782-4f99-9e26-eead67c02682,False,85.869003,5.902000,0.373917,{'d6b293c7-a5d7-4f80-9296-7676eb07cd7b': {'pro...,NONE,91.771004,True,26.0,0.111111,1,target,0,0.0,NaN,NaN


In [680]:
featuresDict

{'atom': array([[0.51190476, 1.        ],
        [0.59433962, 1.        ],
        [0.76470588, 0.        ],
        [0.63636364, 1.        ]]),
 'lm': array([[147.684],
        [116.615]], dtype=float32),
 'lo': array([[1.],
        [1.],
        [1.],
        [1.]]),
 'target': array([[0.11111111]])}

In [681]:
edgesDict

{'atom': array([[0., 2.],
        [1., 3.]]),
 'atom_lm': array([[1.],
        [1.]]),
 'lm_atom': array([[0., 1.],
        [0., 2.]]),
 'atom_target': array([[3.],
        [0.]]),
 'lo_lm': array([[1., 0.],
        [0., 1.]]),
 'lo_atom': array([[1., 1., 0., 0.],
        [0., 1., 2., 3.]]),
 'lo_target': array([[1.],
        [0.]]),
 'lo': array([[0, 3, 1],
        [3, 2, 0]])}

In [682]:
def generate_data_with_target(features,edges,targetCorrectness):  
    data = HeteroData()
    #node features
    data['atom'].x=torch.tensor(features['atom'].astype(np.float64)).float()
    data['lo'].x= torch.tensor(features['lo']).float()
    data['target'].x= torch.tensor(features['target']).float()
    
    #edge index
    data['lo','lo_atom','atom'].edge_index=torch.from_numpy(edges['lo_atom']).long()
    data['lo','lo_lo','lo'].edge_index= torch.from_numpy(edges['lo']).long()
    data['lo','lo_target','target'].edge_index=torch.from_numpy(edges['lo_target']).long()
    data['atom','atom_atom','atom'].edge_index=torch.from_numpy(edges['atom']).long()
    data['atom','atom_target','target'].edge_index=torch.from_numpy(edges['atom_target']).long()
    
    
    
    #lm edges and features
    if 'lm' in features:
        data['lm'].x= torch.tensor(features['lm']).float()
        data['lo','lo_lm','lm'].edge_index=torch.from_numpy(edges['lo_lm']).long()
        
    if 'lm_lm' in features:    
        data['lm','lm_lm','lm'].edge_index=torch.from_numpy(edges['lm_lm']).long()
    
    if 'atom_lm' in features:
        data['atom','atom_lm','lm'].edge_index=torch.from_numpy(edges['atom_lm']).long()
    
    if 'lm_atom' in features:
        data['lm','atom_lm','atom'].edge_index=torch.from_numpy(edges['lm_atom']).long()
    
    data['target'].y=torch.from_numpy(targetCorrectness).long()

    #train_mask = torch.ones(attom_count, dtype=torch.bool)
    #train_mask[target_atom] = False
    #data['atom'].train_mask = train_mask

    normalize=T.NormalizeFeatures()
    data=normalize(data)
    return data

In [683]:
edgesDict

{'atom': array([[0., 2.],
        [1., 3.]]),
 'atom_lm': array([[1.],
        [1.]]),
 'lm_atom': array([[0., 1.],
        [0., 2.]]),
 'atom_target': array([[3.],
        [0.]]),
 'lo_lm': array([[1., 0.],
        [0., 1.]]),
 'lo_atom': array([[1., 1., 0., 0.],
        [0., 1., 2., 3.]]),
 'lo_target': array([[1.],
        [0.]]),
 'lo': array([[0, 3, 1],
        [3, 2, 0]])}

In [684]:
data=generate_data_with_target(featuresDict,edgesDict,targetCorrectness)

In [715]:
model = GNN(hidden_channels=100, out_channels=2)
model = to_hetero(model, data.metadata(), aggr='sum')

learning_rate=0.01
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion=torch.nn.CrossEntropyLoss()
train_acc_mean_list=[]
test_acc_mean_list=[]
loss_list=[]
best_score=0

In [722]:
for i in range(5):
    model.train()
    optimizer.zero_grad()

    out=model(data.x_dict, data.edge_index_dict) 

    loss=criterion(out['target'],data['target'].y[0])
    loss.backward()
    optimizer.step()
    